In [3]:
import datetime as dtm
import requests
import pandas as pd
import time
import os
import numpy as np
import scipy as sp
import pickle
from xgboost import XGBRegressor, XGBClassifier
from xgboost import plot_importance
import matplotlib.pyplot as plt
import src.data_retriever as intrinio
import src.pso as pso
import src.trading_bot as bot
import random
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
%matplotlib inline

In [4]:
random.seed(42)
datefrom = dtm.datetime(2010, 1, 1)
dateto = dtm.datetime(2019, 3, 1)

In [5]:
#DOW 30
# stock_names = ['AAPL', 'CSCO', 'WMT', 'MCD', 'GS', 'AXP', 'BA', 'KO', 'V', 'WMT', 'UNH']
stock_names = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'XOM', 'GS',
               'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
               'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']
# stock_names = ['AAPL']

In [6]:
stocks_filename = 'stocks_dow30_full.csv'
if os.path.isfile(stocks_filename):
    stocks = pd.read_csv(stocks_filename)
    stocks['date'] = pd.to_datetime(stocks['date'], format="%Y-%m-%d")
else:
    stocks = intrinio.get_stocks_data(stock_names, datefrom, dateto)
    stocks.to_csv(stocks_filename, index_label='id', columns=list(stocks.columns))

### Подготовка к тестированию

In [7]:
train_datefrom = datefrom
train_dateto = dtm.datetime(2017, 1, 1)

valid_datefrom = train_dateto
valid_dateto = dtm.datetime(2018, 1, 1)

test_datefrom = dtm.datetime(2018, 1, 1)
test_dateto = dtm.datetime(2019, 1, 1)

In [8]:
data = stocks.drop(0).reset_index(drop=True)
data['date'] = data['date'].map(lambda x: x.timestamp())
#data.drop('date', axis=1)
#test_data = data[data['stock_name'] == 'PFE']
#data = data[data['stock_name'] != 'KO']
#data = data.drop('stock_name', axis=1)
#test_data = test_data.drop('stock_name', axis=1)
#test_data = test_data[test_data['date'] > test_datefrom.timestamp()]

In [9]:
#to_normalize = ['volume_prev1', 'ebitda']
#data['date_norm'] = MinMaxScaler().fit_transform(data['date'].as_matrix().reshape(-1, 1))
#data['volume_prev1'] = MinMaxScaler().fit_transform(data['volume_prev1'].as_matrix().reshape(-1, 1))
#data['ebitda'] = MinMaxScaler().fit_transform(data['ebitda'].as_matrix().reshape(-1, 1))

lbl_encoder=LabelEncoder()
data['stock_name'] = lbl_encoder.fit_transform(data['stock_name'].as_matrix().reshape(-1, 1))
#test_data['stock_name'] = lbl_encoder.transform(test_data['stock_name'].as_matrix().reshape(-1, 1))
# test_data = data[data['stock_name'] == 0]
# data = data[data['stock_name'] != 0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
#lbl_encoder.classes_

In [11]:
#lbl_encoder.inverse_transform(np.array([1]))

In [12]:
train_data = data[data['date'] <= train_dateto.timestamp()]
X_train = train_data.drop(['date','close','adj_close', 'volume'],axis=1)
print(len(X_train))
Y_train_regr = train_data['close']
Y_train_classify = train_data['close'] > train_data['open']

49336


In [13]:
valid_data = data[data['date'] > valid_datefrom.timestamp()]
valid_data = valid_data[valid_data['date'] < valid_dateto.timestamp()]
# valid_data = valid_data.query('(stock_name == 0)')
X_valid = valid_data.drop(['date','close','adj_close', 'volume'],axis=1)
print(len(X_valid))
Y_valid_regr = valid_data['close']
Y_valid_classify = valid_data['close'] > valid_data['open']

7028


In [14]:
test_data = data[data['date'] > test_datefrom.timestamp()]
test_data = test_data[test_data['date'] < test_dateto.timestamp()]
coca_label = lbl_encoder.transform(np.array(['KO']))[0]
intel_label = lbl_encoder.transform(np.array(['INTC']))[0]
pfizer_label = lbl_encoder.transform(np.array(['PFE']))[0]
test_data = test_data[test_data['stock_name'].isin((coca_label, intel_label, pfizer_label))]
# test_data = test_data[test_data['stock_name'] == lbl_encoder.transform(np.array(['UNH']))[0]]
# test_data = test_data[test_data['stock_name'] == lbl_encoder.transform(np.array(['MMM']))[0]]
len(test_data)

753

In [60]:
ts = time.time()
class_params = {
  'colsample_bytree': 0.5908005303400712,
  'eta': 0.13024486152650036,
  'max_depth': 36,
  'min_child_weight': 63,
  'n_estimators': 2553,
  'subsample': 0.31500310724086783}
model_class = XGBClassifier(**class_params)

model_class.fit(
        X_train, 
        Y_train_classify, 
        eval_set=[(X_train, Y_train_classify), (X_valid, Y_valid_classify)], 
        verbose=True,         
        eval_metric='auc',
        early_stopping_rounds = 10)
    
time.time() - ts

[0]	validation_0-auc:0.660703	validation_1-auc:0.650129
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.697907	validation_1-auc:0.680452
[2]	validation_0-auc:0.70801	validation_1-auc:0.686675
[3]	validation_0-auc:0.714658	validation_1-auc:0.692094
[4]	validation_0-auc:0.722554	validation_1-auc:0.699854
[5]	validation_0-auc:0.730637	validation_1-auc:0.706593
[6]	validation_0-auc:0.735304	validation_1-auc:0.712223
[7]	validation_0-auc:0.738228	validation_1-auc:0.716534
[8]	validation_0-auc:0.742408	validation_1-auc:0.718172
[9]	validation_0-auc:0.745058	validation_1-auc:0.722529
[10]	validation_0-auc:0.746179	validation_1-auc:0.721615
[11]	validation_0-auc:0.747988	validation_1-auc:0.7235
[12]	validation_0-auc:0.749616	validation_1-auc:0.725476
[13]	validation_0-auc:0.750665	validation_1-auc:0.727574
[14]	validation_0-auc:0.75198	validation_1-auc:0.727786
[1

16.404703617095947

In [55]:
test_datefrom

datetime.datetime(2018, 1, 1, 0, 0)

In [56]:
test_dateto

datetime.datetime(2019, 1, 1, 0, 0)

In [57]:
def strategy(row):
    strategy_input = pd.DataFrame([row])
    strategy_input = strategy_input.drop(['date','close','adj_close', 'volume'], axis=1)
    decision = 'buy' if model_class.predict(strategy_input) else 'sell'
    return decision

cash_res = bot.simulate_trading(strategy, test_data, initial_cash=400, multipl=10, interest_percent=0.05)
print(cash_res)

sell: open: 36.38, cash=399.21780000000007,  diff=0.5999999999999517, interest=0.1822
buy: open: 36.45, cash=401.63555, diff=2.59999999999998, interest=0.18225
sell: open: 36.71, cash=400.65160000000003,  diff=0.799999999999983, interest=0.18395
sell: open: 36.82, cash=400.06730000000005,  diff=0.3999999999999915, interest=0.18430000000000002
sell: open: 36.72, cash=402.58505,  diff=-2.69999999999996, interest=0.18225
sell: open: 36.44, cash=402.70300000000003,  diff=-0.30000000000001137, interest=0.18205
buy: open: 36.34, cash=403.8213, diff=1.2999999999999545, interest=0.18170000000000003
sell: open: 36.64, cash=404.4385,  diff=-0.799999999999983, interest=0.18280000000000002
sell: open: 36.63, cash=405.1558,  diff=-0.9000000000000341, interest=0.1827
buy: open: 36.57, cash=405.27295000000004, diff=0.30000000000001137, interest=0.18284999999999998
sell: open: 36.82, cash=401.48705000000007,  diff=3.5999999999999943, interest=0.1859
sell: open: 37.17, cash=403.10210000000006,  diff=-1

buy: open: 35.8, cash=502.9458999999999, diff=-0.799999999999983, interest=0.179
buy: open: 35.68, cash=505.66749999999985, diff=2.8999999999999915, interest=0.1784
buy: open: 35.96, cash=504.78769999999986, diff=-0.7000000000000028, interest=0.17980000000000002
sell: open: 35.82, cash=506.0092999999999,  diff=-1.4000000000000057, interest=0.1784
buy: open: 35.37, cash=509.13244999999995, diff=3.300000000000054, interest=0.17685
buy: open: 35.95, cash=509.9526999999999, diff=0.9999999999999432, interest=0.17975
buy: open: 35.95, cash=509.5729499999998, diff=-0.20000000000003126, interest=0.17975
sell: open: 36.26, cash=509.4916999999998,  diff=-0.0999999999999801, interest=0.18125
sell: open: 36.41, cash=508.90944999999977,  diff=0.40000000000006253, interest=0.18225
sell: open: 36.51, cash=509.4272499999998,  diff=-0.7000000000000028, interest=0.1822
sell: open: 36.395, cash=509.3953499999998,  diff=-0.15000000000000568, interest=0.1819
buy: open: 36.49, cash=509.21289999999976, diff=

sell: open: 44.36, cash=643.9194189999995,  diff=-15.499999999999972, interest=0.21405000000000002
buy: open: 42.77, cash=653.8055689999994, diff=10.09999999999998, interest=0.21385000000000004
sell: open: 43.65, cash=658.8899689999994,  diff=-5.300000000000011, interest=0.21559999999999999
buy: open: 43.35, cash=664.4732189999994, diff=5.799999999999983, interest=0.21675
buy: open: 43.77, cash=672.2543689999994, diff=7.999999999999972, interest=0.21885000000000002
sell: open: 44.34, cash=675.4343689999995,  diff=-3.400000000000034, interest=0.22
buy: open: 44.05, cash=679.7141189999995, diff=4.500000000000028, interest=0.22025
buy: open: 44.44, cash=678.7919189999996, diff=-0.7000000000000028, interest=0.22219999999999998
buy: open: 43.89, cash=680.6724689999995, diff=2.1000000000000085, interest=0.21945
sell: open: 43.81, cash=693.7600689999996,  diff=-13.300000000000054, interest=0.21239999999999998
buy: open: 42.39, cash=703.7481189999995, diff=10.19999999999996, interest=0.21195
s

buy: open: 48.57, cash=1062.732543999999, diff=11.799999999999997, interest=0.24285
buy: open: 50.16, cash=1067.981743999999, diff=5.500000000000043, interest=0.25079999999999997
buy: open: 50.18, cash=1079.1308439999991, diff=11.400000000000006, interest=0.2509
sell: open: 51.79, cash=1089.3771439999991,  diff=-10.499999999999972, interest=0.25370000000000004
buy: open: 51.27, cash=1098.320793999999, diff=9.199999999999946, interest=0.25635
buy: open: 51.86, cash=1094.661493999999, diff=-3.399999999999963, interest=0.25930000000000003
buy: open: 52.52, cash=1086.998893999999, diff=-7.40000000000002, interest=0.26260000000000006
buy: open: 52.06, cash=1084.738593999999, diff=-2.0000000000000284, interest=0.26030000000000003
buy: open: 52.23, cash=1070.977443999999, diff=-13.499999999999943, interest=0.26115
buy: open: 51.17, cash=1070.721593999999, diff=0.0, interest=0.25585
buy: open: 50.74, cash=1071.3678939999988, diff=0.899999999999963, interest=0.25370000000000004
buy: open: 50.88

sell: open: 52.76, cash=1324.0319249999995,  diff=-5.799999999999983, interest=0.26089999999999997
sell: open: 52.21, cash=1321.5697749999995,  diff=2.1999999999999886, interest=0.26215
buy: open: 52.45, cash=1318.4075249999994, diff=-2.9000000000000625, interest=0.26225
buy: open: 48.76, cash=1307.3637249999995, diff=-10.799999999999983, interest=0.2438
sell: open: 47.63, cash=1306.5252749999995,  diff=0.5999999999999517, interest=0.23845
buy: open: 48.24, cash=1304.8840749999995, diff=-1.4000000000000057, interest=0.24120000000000003
buy: open: 48.055, cash=1312.1937999999996, diff=7.550000000000026, interest=0.24027500000000002
buy: open: 48.5, cash=1321.7512999999994, diff=9.799999999999969, interest=0.2425
buy: open: 49.5, cash=1322.8037999999995, diff=1.3000000000000256, interest=0.2475
buy: open: 48.77, cash=1327.8599499999993, diff=5.29999999999994, interest=0.24385000000000004
buy: open: 49.43, cash=1330.3127999999995, diff=2.7000000000000313, interest=0.24715
buy: open: 49.6,

buy: open: 47.47, cash=1743.012174999999, diff=2.7000000000000313, interest=0.23735
sell: open: 47.49, cash=1761.984324999999,  diff=-19.200000000000017, interest=0.22785
sell: open: 45.41, cash=1760.456624999999,  diff=1.3000000000000256, interest=0.22769999999999999
sell: open: 45.75, cash=1769.332424999999,  diff=-9.099999999999966, interest=0.2242
sell: open: 44.63, cash=1779.5144749999988,  diff=-10.399999999999991, interest=0.21795000000000003
buy: open: 44.0, cash=1801.1944749999989, diff=21.899999999999977, interest=0.22
sell: open: 45.62, cash=1793.5626749999988,  diff=7.40000000000002, interest=0.2318
sell: open: 46.51, cash=1790.9289249999988,  diff=2.40000000000002, interest=0.23375
sell: open: 47.09, cash=1792.2942749999988,  diff=-1.600000000000037, interest=0.23465
sell: open: 45.91, cash=1795.7665749999987,  diff=-3.6999999999999744, interest=0.22769999999999999
sell: open: 45.49, cash=1796.0393749999987,  diff=-0.5000000000000426, interest=0.22719999999999999
buy: open

sell: open: 42.24, cash=1939.3304249999978,  diff=-1.0000000000000142, interest=0.2107
sell: open: 42.25, cash=1941.2202249999978,  diff=-2.1000000000000085, interest=0.2102
sell: open: 41.78, cash=1941.6116249999977,  diff=-0.6000000000000227, interest=0.2086
sell: open: 41.74, cash=1943.3038749999978,  diff=-1.9000000000000483, interest=0.20775000000000002
sell: open: 42.01, cash=1940.1923749999978,  diff=2.8999999999999915, interest=0.2115
sell: open: 42.3, cash=1941.1814749999978,  diff=-1.1999999999999744, interest=0.2109
sell: open: 42.31, cash=1939.969424999998,  diff=0.9999999999999432, interest=0.21205
sell: open: 42.5, cash=1942.258174999998,  diff=-2.5, interest=0.21125
sell: open: 42.25, cash=1940.546174999998,  diff=1.4999999999999858, interest=0.212
sell: open: 42.4, cash=1941.134574999998,  diff=-0.799999999999983, interest=0.2116
buy: open: 42.34, cash=1941.5228749999978, diff=0.5999999999999517, interest=0.21170000000000003
buy: open: 42.12, cash=1946.9122749999979, di

buy: open: 45.66, cash=2056.3297249999973, diff=1.9000000000000483, interest=0.22829999999999998
buy: open: 45.94, cash=2055.5000249999975, diff=-0.5999999999999517, interest=0.2297
buy: open: 45.99, cash=2060.1700749999977, diff=4.899999999999949, interest=0.22995000000000002
buy: open: 46.38, cash=2060.038174999998, diff=0.0999999999999801, interest=0.23190000000000002
sell: open: 46.3, cash=2066.0097749999977,  diff=-6.199999999999974, interest=0.22840000000000002
sell: open: 45.8, cash=2077.3865749999977,  diff=-11.599999999999966, interest=0.22319999999999998
sell: open: 44.48, cash=2075.163174999998,  diff=2.0000000000000284, interest=0.22340000000000002
buy: open: 44.64, cash=2077.639974999998, diff=2.69999999999996, interest=0.22319999999999998
buy: open: 45.02, cash=2082.114874999998, diff=4.699999999999989, interest=0.22510000000000002
buy: open: 45.41, cash=2086.387824999998, diff=4.500000000000028, interest=0.22704999999999997
sell: open: 45.9, cash=2089.059774999998,  diff